# **Reinforcement Learning: Taller 4**
## Estudiantes: Juan Pablo Reyes Fajardo y Santiago Rodríguez Ávila 

In [2]:
!pip install gym

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('CartPole-v1')
cart_high_var = env.observation_space.high
cart_low_var = env.observation_space.low

def num_states(num):
    # dependiendo el numero de estados, cambia el tamaño de cada vector
    if num ==400:
        car_pos = np.linspace(cart_low_var[0], cart_high_var[0], num= 5)
        car_vel = np.linspace(-2, 2, num= 4)
        pole_angle = np.linspace(cart_low_var[2], cart_high_var[2], num = 5)
        pole_vel = np.linspace(-2, 2, num= 4)
    elif num ==4000:
        car_pos = np.linspace(cart_low_var[0], cart_high_var[0], num= 8)
        car_vel = np.linspace(-2, 2, num= 8)
        pole_angle = np.linspace(cart_low_var[2], cart_high_var[2], num = 8)
        pole_vel = np.linspace(-2, 2, num= 8)   
    # vector donde estan todos los valores que se pueden colocar en una tabla
    discrete_values = [car_pos, car_vel, pole_angle, pole_vel]
    return discrete_values


discrete_values = num_states(400)
Q_table = {}
states=[]
def set_states():
    for cpos in discrete_values[0]:
        for cvel in discrete_values[1]:
            for pang in discrete_values[2]:
                for pvel in discrete_values[3]:
                    states.append((cpos, cvel, pang, pvel))
                    
def set_q_table():
    for i in states:
        for action in range(2):
            Q_table[i,action] = 0
    
                    
def discretize(new_state):
    # car pos, car vel, pole angle, pole vel
    valor_cercano=[0]*4
    for i in range(4):
        dif = [(abs(x - new_state[i])) for x in discrete_values[i]]
        valor_cercano[i] = discrete_values[i][dif.index(min(dif))]
        
    return valor_cercano

In [5]:
#SARSA algorithm
def SARSA_algorithm(num_episodes):
    allowed_actions=[0,1]
    # Variables: alpha, gamma y epsilon.
    alpha = 0.3
    gamma = 1
    epsilon = 0.1
    cont = 0
    #Para 5000 episodios de entrenamiento
    for i in range(num_episodes):
        # Inicializa las variables para cada episodio
        state=env.reset()
        num_steps = 0
        state = discretize(state[0])


        # Seleccion accion "a" de forma epsilon-greedy
        if epsilon< np.random.uniform():        
            act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
            action = allowed_actions[np.argmax(act_arg)]
        else:
            action = env.action_space.sample()

        # Inicia el episodio
        continue_episode = True
        while continue_episode:        
            # Obtengo s'
            new_state, reward, done, _,_=env.step(action)
            new_state = discretize(new_state)

            # Revisa que new_state no sea un estado terminal
            if done:
                # Valor q(s',a') terminal
                q_value_next_step = 0
            else:
                #Obtengo a' de s' con epsilon greedy
                if epsilon< np.random.uniform():        
                    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
                    new_action = allowed_actions[np.argmax(act_arg)]
                else:
                    new_action = env.action_space.sample()

                # Valor q(s',a') no terminal
                q_value_next_step = Q_table[tuple(new_state),new_action]


            # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*q(s',a') - q(s,a))
            Q_table[tuple(state), action] += alpha*(reward + gamma*q_value_next_step - Q_table[tuple(state),action])

            # asigna a = a' y s = s'
            state = new_state
            action = new_action


            # Parte que termina el episodio si se llega a algun estado terminal
            if done:
                continue_episode = False
        cont+=1
        if cont%500==0:
            print(cont)

## SARSA con 400 estados:

In [4]:
set_states()
set_q_table()

In [5]:
discrete_values = num_states(400)
SARSA_algorithm(15000)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000


In [18]:
allowed_actions= [0,1]
prom=[]
prom_rand=[]
for i in range(100):
    G_pi = 0
    state =env.reset()
    state = state[0]
    state = discretize(state)
    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
    action = allowed_actions[np.argmax(act_arg)]
    done = False
    n=1
    while not done:
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
        discrete_state = discretize(new_state)
        act_arg = np.array([Q_table[tuple(discrete_state), act] for act in range(2)])
        action = allowed_actions[np.argmax(act_arg)]
    prom.append(G_pi)
print('---'*5)
for i in range(100):
    G_pi = 0
    state =env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
    prom_rand.append(G_pi)

---------------


In [19]:
print("Retorno obtenido con la funcion q:",sum(prom)/len(prom))
print("Retorno obtenido de acciones al azar:",sum(prom_rand)/len(prom_rand))

Retorno obtenido con la funcion q: 47.26
Retorno obtenido de acciones al azar: 24.58


## SARSA con 4000 estados:

In [7]:
set_states()
set_q_table()

In [11]:
discrete_values = num_states(4000)
SARSA_algorithm(5000)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [12]:
prom=[]
prom_rand=[]
allowed_actions= [0,1]
for i in range(100):
    G_pi = 0
    state =env.reset()
    state = state[0]
    state = discretize(state)
    act_arg = np.array([Q_table[tuple(state), act] for act in range(2)])
    action = allowed_actions[np.argmax(act_arg)]
    done = False
    n=1
    while not done:
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
        discrete_state = discretize(new_state)
        act_arg = np.array([Q_table[tuple(discrete_state), act] for act in range(2)])
        action = allowed_actions[np.argmax(act_arg)]
    prom.append(G_pi)
print('---'*5)
for i in range(100):
    G_pi = 0
    state =env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        new_state, reward, done, _,_=env.step(action)
        G_pi= G_pi + reward
    prom_rand.append(G_pi)

---------------


In [13]:
print("Retorno obtenido con la funcion q:",sum(prom)/len(prom))
print("Retorno obtenido de acciones al azar:",sum(prom_rand)/len(prom_rand))

Retorno obtenido con la funcion q: 197.99
Retorno obtenido de acciones al azar: 22.65


In [20]:
env.close()

## Uso de redes neuronales

In [14]:
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
'''
# Define the hyperparameters
EPISODES = 5000
EPSILON = 1.0
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.999
GAMMA = 0.99
ALPHA = 0.001
BATCH_SIZE = 32

# Define the environment
env = gym.make('CartPole-v1')

# Define the model
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam')
'''



class DSARSAAgent:
    def __init__(self, state_dim, action_dim, learning_rate=0.001, discount_factor=0.99, epsilon=1.0, epsilon_decay=0.999):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(4, input_dim=self.state_dim, activation='relu'))
        model.add(Dense(2, activation='linear'))
        model.compile(loss='mse', optimizer='adam')
        print(model.summary())
        return model

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_dim)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def update_model(self, state, action, reward, next_state, next_action, done):
        target = reward
        if not done:
            target += self.discount_factor * self.model.predict(next_state)[0][next_action]
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)

In [13]:
env = gym.make('CartPole-v1',render_mode="human")
num_episodes=100
# Define your state and action dimensions
state_dim = 4
action_dim = 2

# Create an instance of the DSARSAAgent class
agent = DSARSAAgent(state_dim, action_dim)
for episode in range(num_episodes):
    state = env.reset()
    state = state[0]
    done = False
    total_reward = 0

    while not done:
        # Choose an action
        action = agent.choose_action(state)

        # Take the chosen action and observe the next state and reward
        next_state, reward, done, _,_ = env.step(action)
        next_state = np.reshape(next_state, [1, state_dim])

        # Choose the next action
        next_action = agent.choose_action(next_state)

        # Update the model
        agent.update_model(state, action, reward, next_state, next_action, done)

        # Update the current state
        state = next_state
        total_reward += rewardsummary

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 4)                 20        
                                                                 
 dense_15 (Dense)            (None, 2)                 10        
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 362ms/step


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_7/dense_14/MatMul' defined at (most recent call last):
    File "C:\Users\Santiago\miniconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Santiago\miniconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Santiago\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Santiago\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Santiago\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 724, in start
      self.io_loop.start()
    File "C:\Users\Santiago\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\Santiago\miniconda3\lib\asyn

In [14]:
env.close()

In [4]:
!pip install keras-rl

     -------------------------------------- 40.5/40.5 kB 642.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-rl: filename=keras_rl-0.4.2-py3-none-any.whl size=48362 sha256=1d65f8ddb311e3664a0f4eed182532dca4fa4befdd6df148768defe288c7f8c1
  Stored in directory: c:\users\santiago\appdata\local\pip\cache\wheels\12\c5\34\a89a10839a1fe8f3e38fe7c9f81faa3f244b55c3b9a04a5b34
Successfully built keras-rl


In [16]:
from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy

# Define the environment
env = gym.make('CartPole-v1')
np.random.seed(123)
nb_actions = env.action_space.n

# Define the model
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
model.add(Dense(2, activation='linear'))
#model.compile(loss='mse', optimizer='adam')

policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(learning_rate=1e-3), metrics=['mae'])

#sarsa.fit(env, nb_steps=100, visualize=False, verbose=2)

# After training is done, we save the final weights.
#sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)


                   
        
#weights=model.get_weights()
#for i, layer_weights in enumerate(weights):
    #print(f"Capa {i + 1} - Pesos:")
    #print(layer_weights)
    #print()

In [20]:
sarsa.fit(env, nb_steps=100, visualize=False, verbose=1)

Training for 100 steps ...
Interval 1 (0 steps performed)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=True)

In [4]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy


ENV_NAME = 'CartPole-v1'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
states= 
np.random.seed(123)
#env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(4))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(learning_rate=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=1000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
 activation_2 (Activation)   (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 2)                 10        
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
None
Training for 1000 steps ...


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [1]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy

env = gym.make('CartPole-v1')
states = env.observation_space.shape
actions = env.action_space.n

def build_model(states, actions):
    model = Sequential()
    
    model.add(Dense(4, activation = 'relu',input_shape= states))
    model.add(Dense(actions, activation = 'linear'))
    return model

model = build_model(states,actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Agente
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model,memory=memory,  policy=policy,
                   nb_actions= actions, nb_steps_warmup=10, target_model_update = 1e-2)
    return dqn
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=1000,visualize=False,verbose = 1)

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.